In [1]:
!hdfs dfs -tail /labs/slaba02/DO_record_per_line.json

rkable medical revolution going on today. Students will complete this course in 12 weeks. Each week features a series of lecture sequences that are supported by interactive video tutorials and interspersed exercises or problems.&nbsp;Students will work on a homework assignment or exam each week. The course will conclude with a comprehensive final exam. 7.00x \u2013 The Secret of Life will let you explore the mysteries of biochemistry, genetics, molecular biology, recombinant DNA technology and genomics, and rational medicine. We are excited to take this journey with you! Before your course starts, try the new edX Demo where you can explore the fun, interactive learning environment and virtual labs.&nbsp;Learn more. FAQ Can I still choose an honor code certificate at no cost? Yes, if you want an Honor Code certificate after successfully completing the course and do not need or want the verified certificate, you can select that option by clicking \u201cWhy do I have to pay?\u201d under t

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "laba2 app") 

spark = SparkSession.builder.config(conf=conf).appName("laba2").getOrCreate()

In [4]:
spark

In [4]:
sc = spark.sparkContext

In [9]:
df = spark.read.json("/labs/slaba02/DO_record_per_line.json")

In [11]:
df.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
+--------------------+--------------------+---+----+--------------------+--------------+
only showing top 5 rows



In [10]:
# курс

In [11]:
id = [23126, 21617, 16627, 11556, 16704, 13702]

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, lit

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

import json

In [ ]:
# не плохое дополнение!

# stop_words = StopWordsRemover.loadDefaultStopWords("english")
# swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_filtered", stopWords=stop_words)

# preprocessing = Pipeline(stages=[
#     tokenizer,
#     swr,
#     count_vectorizer
# ])

# preprocessing_model = preprocessing.fit(new_df)
# preprocessed_dataset = preprocessing_model.transform(new_df)

In [229]:
dict_mas = {}
for i in [23126, 21617, 16627, 11556, 16704, 13702]:
    lang = df.filter('id == ' + str(i)).select('lang').rdd.flatMap(lambda x: x).collect()[0]
    new_df = df.filter(df.lang.like(lang))

    # TFIDF
    tokenizer = Tokenizer(inputCol="desc", outputCol="words")
    wordsData = tokenizer.transform(new_df)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    # Нормализация
    normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=2.0)
    l2NormData = normalizer.transform(rescaledData)

    # добавление колонки с id заданого вектора
    v = l2NormData.filter('id == ' + str(i)).select(col('normFeatures').alias("id_vector")).limit(1)
    new_df = l2NormData.crossJoin(v)

    # cos расстояние
    dotProductUDF = udf(lambda v1, v2: float(v1.dot(v2)), DoubleType())
    new_df = new_df.withColumn("cos", dotProductUDF("id_vector", "normFeatures"))[['id', 'cos']]

    new_df = new_df[new_df['id'] != i]
    new_df = new_df[new_df['cos'] < 0.99]
    new_df = new_df.orderBy(new_df['cos'].desc())
    print('#####################')
    print(i)
    print(new_df.show(10))

    mas = new_df.select('id').limit(10).collect()
    mas = [j['id'] for j in mas]
    dict_mas[str(i)] = mas

#####################
23126
+-----+-------------------+
|   id|                cos|
+-----+-------------------+
|13782| 0.4720272349196929|
|13665|0.44990966856587233|
|24419|0.42697047829852686|
|20638| 0.4106274562765083|
| 2724| 0.3797487681859415|
|25782| 0.2837238538640951|
| 2633|0.26715960070589767|
| 2723| 0.2657748793766494|
|13348| 0.2504132200626535|
|15909|0.24576952252068635|
+-----+-------------------+
only showing top 10 rows

None
#####################
21617
+-----+-------------------+
|   id|                cos|
+-----+-------------------+
|21608| 0.4684746966615021|
|21616| 0.4645572985597666|
|21492|0.37345437787709834|
|21624| 0.3191321071809383|
|21623|0.31407486478991886|
|21630|0.31390543251576386|
|21628|0.31380989052004515|
|21508| 0.3058525152715214|
|21857| 0.3046877707105122|
|21506|0.30304327691152194|
+-----+-------------------+
only showing top 10 rows

None
#####################
16627
+-----+-------------------+
|   id|                cos|
+-----+-------

In [230]:
variant1 = dict_mas.copy()
variant1

{'23126': [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909],
 '21617': [21608,
  21616,
  21492,
  21624,
  21623,
  21630,
  21628,
  21508,
  21857,
  21506],
 '16627': [11431, 12247, 5687, 17964, 12660, 16694, 17961, 9563, 5558, 9598],
 '11556': [16488, 10384, 22710, 468, 10447, 13461, 19330, 23357, 12679, 21707],
 '16704': [1228, 1219, 20362, 1327, 1365, 55, 26980, 18331, 1236, 8186],
 '13702': [21079, 792, 8123, 8083, 1033, 1041, 13057, 19925, 1217, 1396]}

In [235]:
with open("/data/home/yakov.zakharov/lab02.json", "w") as write_file:
    json.dump(variant1, write_file)